<p style="font-family: Cambria; text-align: center; font-size: 48px;">  I. DATA PRE-PROCESSING</p>

In [1]:
#Importing all the Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
#Reading the file schema_1_ontario_final
df_schema1=pd.read_csv("schema_1_ontario_final.csv")

In [3]:
#Reading the file schema_2_ontario_final
df_schema2=pd.read_csv("schema_2_ontario_final.csv")

In [4]:
#Reading the file schema_3_ontario_final
df_schema3=pd.read_csv("schema_3_ontario_final.csv")

In [5]:
#combine 3 schema files into a single file
df_combined = pd.concat([df_schema1, df_schema2, df_schema3], ignore_index=True, sort=False)

In [6]:
df_combined.to_csv("covid_python_Dec25_beforecleaning.csv",index=False,encoding="utf-8-sig")

## <b style="font-family: Cambria; font-size: 16px;">Renaming age_1 to age_category as this column has age category rather than age<b>

In [7]:
df_combined.rename(columns={'age_1': 'age_category'}, inplace=True)

## <b style="font-family: Cambria; font-size: 16px;">After concatenating the three schema files, the columns which are not present in any one of the files has null values.These nulls are set to NR - 'Not Recorded' as these columns were not part of that particular survey response.<b>

In [8]:
df_combined['week'] = df_combined['week'].fillna('NR')
df_combined['month'] = df_combined['month'].fillna('NR')
df_combined['symptoms'] = df_combined['symptoms'].fillna('NR')
df_combined['conditions'] = df_combined['conditions'].fillna('NR')
df_combined['ethnicity'] = df_combined['ethnicity'].fillna('NR')
df_combined['sex'] = df_combined['sex'].fillna('NR')
df_combined['needs'] = df_combined['needs'].fillna('NR')
df_combined['age_category'] = df_combined['age_category'].fillna('NR')
df_combined['contact_in_household'] = df_combined['contact_in_household'].fillna('NR')
df_combined['tested'] = df_combined['tested'].fillna('NR')
df_combined['covid_results_date'] = df_combined['covid_results_date'].fillna('NR')
df_combined['covid_positive'] = df_combined['covid_positive'].fillna('NR')
df_combined['mental_health_impact'] = df_combined['mental_health_impact'].fillna('NR')
df_combined['travel_work_school'] = df_combined['travel_work_school'].fillna('NR')
df_combined['self_isolating'] = df_combined['self_isolating'].fillna('NR')
df_combined['media_channels'] = df_combined['media_channels'].fillna('NR')
df_combined['financial_obligations_impact'] = df_combined['financial_obligations_impact'].fillna('NR')
df_combined['tobacco_usage'] = df_combined['tobacco_usage'].fillna('NR')

## <b style="font-family: Cambria; font-size: 16px;">Removing noise in data. 'na' found in the sex column is set to NR<b>

In [9]:
df_combined['sex'] = df_combined['sex'].replace('na', 'NR')

## <b style="font-family: Cambria; font-size: 16px;"> Removing noise in data. Replace 'n' to negatively in the covid_positive column assuming n means negative<b>

In [10]:
df_combined['covid_positive'] = df_combined['covid_positive'].replace('n', 'negatively')

## <b style="font-family: Cambria; font-size: 16px;"> For those records over_60 was not recorded, set over_60 to y if age_category '> 65'<b>

In [11]:
df_combined.loc[df_combined['over_60'].isna() & (df_combined['age_category'] == '>65'), 'over_60'] = 'y'

## <b style="font-family: Cambria; font-size: 16px;"> Create a new column province using the first character of FSA. The first character of FSA is a letter that identifies the province or territory.<b>

In [12]:
fsa_mapping = {
    'K': 'Eastern Ontario',
    'L': 'Central Ontario',
    'M': 'Metropolitan Toronto',
    'N': 'Southwestern Ontario',
    'P': 'Northern Ontario'
}

df_combined['province'] = df_combined['fsa'].astype(str).str[0].map(fsa_mapping)

## <b style="font-family: Cambria; font-size: 16px;">Create a new column region to indicate urban or rural region using second character of FSA. The second character is a numeral that identifies whether the area is urban or rural. A zero indicates a wide-area rural region, while all other digits indicate urban areas.<b>

In [13]:
second_char = df_combined['fsa'].astype(str).str[1]
df_combined['region'] = np.where(second_char == '0', 'Rural', 'Urban')

In [14]:
len(df_combined)

294105

## <b style="font-family: Cambria; font-size: 16px;"> Create a new column, ethnicity_clean, to separate multiple values in the ethnicity field.<b>


In [15]:
# Cleaning function
# -----------------------------
def clean_ethnicity(value):
    if pd.isna(value):
        return np.nan
    
    value = value.strip().lower()
    
    # Handle missing / non-informative values
    if value in ['nr', 'na', 'not reported', 'not available']:
        return 'NR'
    
    # Split by semicolon first
    value = value.split(';')[0]
    
    # If slash exists (e.g. hispanic/latino), keep first
    #value = value.split('/')[0]
    
    return value.title()

# Apply cleaning
df_combined['ethnicity_clean'] = df_combined['ethnicity'].apply(clean_ethnicity)



## <b style="font-family: Cambria; font-size: 16px;">This code uses a mapping dictionary to standardize and clean the travel_work_school column <b> 

In [16]:
# Create a mapping for cleaning
travel_mapping = {
    'NR': 'Not Reported',
    'stoppedTravelling': 'Stopped Traveling',
    'stillTravelEssential': 'Still Travel (Essential)',
    'stillTravelNonEssential': 'Still Travel (Non-Essential)',
    'alwaysWorkFromHome': 'Always Work From Home',
    'didntTravelBefore': 'Did Not Travel Before',
    "non: j'ai toujours travaille de chez moi.": 'Always Work From Home'
}

# Apply the mapping
df_combined['travel_work_school'] = df_combined['travel_work_school'].replace(travel_mapping)


In [17]:
#Write to csv file
df_combined.to_csv("covid_python_Dec25.csv",index=False,encoding="utf-8-sig")